The gaia data is currently in a large list of gzip'd csvs. Why? Because astronomy. I'm glad it's not in fits files, frankly. I'm gonna first try putting in HDF5, which will be a marginal improvement. If that doesn't quite work we can always do a SQL database but that may be more trouble I"m prepared to commit to. 

In [1]:
import numpy as np
from os import path
import pandas as pd
from glob import glob

/home/users/swmclau2/.local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
gaia_dir = '/oak/stanford/orgs/kipac/edarragh/gaiadr2'

In [3]:
ex_file = pd.read_csv(path.join(gaia_dir, 'GaiaSource_2024241623026358656_2024280144613411456.csv.gz'))

In [4]:
data_files = sorted(glob(path.join(gaia_dir, 'GaiaSource*.csv.gz')))

In [5]:
ra_bins = np.linspace(0, 360, 72+1)
dec_bins = np.linspace(-90,90, 36+1)

In [7]:
f  = pd.HDFStore(path.join(gaia_dir, 'gaiadr2.hdf5'), 'w')

In [8]:
k = 0
for d in data_files:
    data = pd.read_csv(d)
    for i, ra in enumerate(ra_bins[:-1]):
        data_ra_slice = data[(data['ra']>=ra) & (data['ra']<ra_bins[i+1])]
        if len(data_ra_slice) == 0:
            continue
        group_key = 'RA_%d_%d/'%(ra, ra_bins[i+1])
        for j, dec in enumerate(dec_bins[:-1]):
            data_ra_dec_slice = data_ra_slice[(data_ra_slice['dec']>=dec)\
                                         & (data_ra_slice['dec']<dec_bins[j+1])]
            if len(data_ra_dec_slice)>0:
                dset_key = 'DEC_%d_%d'%(dec, dec_bins[j+1])
                f.put(group_key+dset_key, data_ra_dec_slice, format='table', append = True)

    k+=1
    print k,
    if k>100:
        break

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101


In [9]:
f.close()